In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [20]:
#read in training data
train_df = pd.read_csv('../data/train_sessions.csv')
test_df = pd.read_csv('../data/test_sessions.csv')

In [6]:
train_df.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [36]:
#read in training data
train_df = pd.read_csv('../data/train_sessions.csv')
test_df = pd.read_csv('../data/test_sessions.csv')

#train_df = train_df.sort_values('time1')


sites = [f'site{i}' for i in range(1, 11)]
train_df = train_df[sites + ['target']].fillna(0).astype(int)
test_df = test_df[sites].fillna(0).astype(int)

y_train = train_df['target']
train_df = train_df.drop('target', axis=1)


train_df.to_csv("../data/train_df.txt", sep=" ", index=False, header=False)
test_df.to_csv("../data/test_df.txt", sep=" ", index=False, header=False)

In [75]:
train_df.shape, test_df.shape

((253561, 10), (82797, 10))

In [27]:
!head ../data/train_df.txt

56 55 0 0 0 0 0 0 0 0
56 55 56 55 0 0 0 0 0 0
946 946 951 946 946 945 948 784 949 946
945 948 949 948 945 946 947 945 946 946
947 950 948 947 950 952 946 951 946 947
952 947 953 946 947 946 953 955 946 947
953 947 946 953 955 947 953 946 953 1033
946 947 954 953 946 954 946 956 957 956
946 956 946 946 955 954 946 946 946 948
948 946 948 784 49 53 812 982 52 52


In [37]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
with open('../data/train_df.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('../data/test_df.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
print(X_train.shape, X_test.shape)

(253561, 50000) (82797, 50000)


In [74]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
def get_auc_lr_valid(X, y, C=0.7, seed=42, ratio = 0.85):
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=500).fit(X[:idx, :], y[:idx])
    # Prediction for validation set
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [34]:
X_train.shape, y_train.shape

((253561, 50000), (253561,))

In [38]:
print(get_auc_lr_valid(X_train, y_train))

0.958792394161128


In [67]:
lr = LogisticRegression(C=0.7, random_state=42, solver='lbfgs', max_iter=500).fit(X_train, y_train)
y_test = lr.predict_proba(X_test)[:,1]
submission = pd.DataFrame(y_test, index=np.arange(1, len(y_test)+1))

In [68]:
submission.columns = ['target']

In [69]:
submission.to_csv('../data/submission_baseline_implementation.csv', index_label='session_id')

In [39]:
!ls ../data

catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2.zip
sample_submission.csv
site_dic.pkl
test_df.txt
test_sessions.csv
test_split.txt
train.zip
train_clean.txt
train_df.txt
train_sessions.csv
train_split.txt


In [40]:
pd.read_csv('../data/sample_submission.csv')

,session_id,target
0,1,0.948255
1,2,0.682483
2,3,0.502855
3,4,0.345556
4,5,0.889428
...,...,...
82792,82793,0.585216
82793,82794,0.621074
82794,82795,0.651144
82795,82796,0.319917


In [61]:
!head ../data/submission_baseline_implementation.csv

session_id,session_id,target
0,1.0,0.002067758305886315
1,2.0,5.951776873354833e-08
2,3.0,1.029799291849713e-07
3,4.0,4.0897644737663995e-07
4,5.0,1.9423949621560488e-05
5,6.0,0.0006119787925679998
6,7.0,0.0017005570004024794
7,8.0,0.00042348562771462613
8,9.0,0.0003908953826742888
